# Trabajar con el CDK de AWS

Anteriormente dijimos que componente básico para una aplicación de CDK era el constructor.
<br>
<br>

![Image](https://docs.aws.amazon.com/images/cdk/v2/guide/images/AppStacks.png)

 * Un constructor representa un componente de la nube e incluye todo lo que AWS CloudFormation necesita para crear el componente. 
 
 * Un constructor puede representar un único recurso de la nube (como una función Lambda), o puede representar un componente de nivel superior formado por varios recursos de AWS.
 
 * Como desarrollador, puede incorporar constructores existentes o crear sus propios constructores. Para crear un nuevo constructor, cree una clase que extienda la clase base del constructor. A continuación, introduzca los parámetros *scope*, *id* y *props* cuando se inicialice el constructor. 


>`rm -rf hello-cdk # Remove this directory if it already exists.` <br>
>`mkdir hello-cdk  # Create the directory.` <br>
>`cd hello-cdk     # Switch to the directory.`

>`cdk init sample-app --language typescript`

Esto crea los siguientes archivos y subdirectorios en el directorio.

* Un subdirectorio .git oculto y un archivo .gitignore oculto, que hace que el proyecto sea compatible con herramientas de control de fuentes como Git.

* Un subdirectorio **lib**, que incluye un archivo hello-cdk-stack.ts. Este archivo contiene el código para su pila AWS CDK. Este código se describe en el siguiente paso de este procedimiento.

* Un subdirectorio **bin**, que incluye un archivo hello-cdk.ts. Este archivo contiene el punto de entrada para su aplicación AWS CDK.

* Un subdirectorio **node_modules**, que contiene paquetes de código de apoyo que la aplicación y la pila pueden utilizar según sea necesario.

* Un archivo oculto **.npmignore**, que enumera los tipos de subdirectorios y archivos que npm no necesita cuando construye el código.

* Un archivo **cdk.json**, que contiene información para facilitar la ejecución del comando cdk.

* Un archivo **package-lock.json**, que contiene información que npm puede utilizar para reducir posibles errores de construcción y ejecución.

* Un archivo *package.json*, que contiene información para hacer que la ejecución del comando npm sea más fácil y posiblemente con menos errores de construcción y ejecución.

* Un archivo *README.md*, que enumera los comandos útiles que puede ejecutar con npm y el CDK de AWS.

* Un archivo **tsconfig.json**, que contiene información para que la ejecución del comando tsc sea más fácil y posiblemente con menos errores de compilación y ejecución.

Abra el archivo lib/hello-cdk-stack.ts, y busque el siguiente código en ese archivo.

In [ ]:
import { Duration, Stack, StackProps } from 'aws-cdk-lib';
import * as sns from 'aws-cdk-lib/aws-sns';
import * as subs from 'aws-cdk-lib/aws-sns-subscriptions';
import * as sqs from 'aws-cdk-lib/aws-sqs';
import { Construct } from 'constructs';

export class HelloCdkStack extends Stack {
  constructor(scope: Construct, id: string, props?: StackProps) {
    super(scope, id, props);

    const queue = new sqs.Queue(this, 'HelloCdkQueue', {
      visibilityTimeout: Duration.seconds(300)
    });

    const topic = new sns.Topic(this, 'HelloCdkTopic');

    topic.addSubscription(new subs.SqsSubscription(queue));
  }
}

* Las clases Stack, App, StackProps, Queue y Topic representan una pila de AWS CloudFormation y sus propiedades, un programa ejecutable, una cola de Amazon SQS y un topic de Amazon SNS, respectivamente.

* La clase HelloCdkStack representa la pila de AWS CloudFormation para esta aplicación. Esta pila contiene la nueva cola de Amazon SQS y el tema de Amazon SNS para esta aplicación.

Abra el archivo bin/hello-cdk.ts, este código carga, instala y ejecuta la clase HelloCdkStack del archivo lib/hello-cdk-stack.ts:

In [ ]:
#!/usr/bin/env node
import * as cdk from 'aws-cdk-lib';
import { HelloCdkStack } from '../lib/hello-cdk-stack';

const app = new cdk.App();
new HelloCdkStack(app, 'HelloCdkStack');

Utilice npm para ejecutar el compilador de TypeScript para comprobar si hay errores de codificación y, a continuación, habilite el CDK de AWS para ejecutar el archivo bin/hello-cdk.js del proyecto. Para ello, desde el directorio raíz del proyecto, ejecute el comando npm con la acción de ejecución, especificando el valor del comando de compilación en el archivo package.json, de la siguiente manera.

> `npm run build`

El comando anterior ejecuta el compilador de TypeScript, que añade los archivos de soporte bin/hello-cdk.d.ts y lib/hello-cdk-stack.d.ts. El compilador también transpila los archivos hello-cdk.ts y hello-cdk-stack.ts a los archivos hello-cdk.js y hello-cdk-stack.js.

*Transpilador es un tipo especial de compilador que traduce de un lenguaje fuente a otro fuente también de un nivel de abstracción parecido. Se diferencia de los compiladores tradicionales en que estos reciben como entrada ficheros conteniendo código fuente y generan código máquina del más bajo nivel. Wikipedia*

### Ejecutar el código

En este paso, se indica al CDK de AWS que cree una plantilla de pila de AWS CloudFormation basada en el código del archivo bin/hello-cdk.js. A continuación, se indica al CDK de AWS que implemente la pila, que crea el tema de Amazon SNS y la cola de Amazon SQS y, a continuación, suscribe la cola al tema. A continuación, confirma que el tema y la cola se han desplegado correctamente enviando un mensaje desde el tema a la cola.

1. Haz que el CDK de AWS cree la plantilla de pila de AWS CloudFormation. Para ello, con la sesión de terminal aún abierta en el IDE, desde el directorio raíz del proyecto, ejecuta el comando cdk con la acción synth y el nombre de la pila. Si tiene éxito, la salida muestra la sección de recursos de la plantilla de AWS CloudFormation.

> `cdk synth HelloCdkStack`

<br>

2. La primera vez que implemente una aplicación AWS CDK en un entorno para una combinación específica de cuenta de AWS y región de AWS, debe instalar una pila de arranque. Esta pila incluye varios recursos que el CDK de AWS necesita para completar sus diversas operaciones. Por ejemplo, esta pila incluye un cubo de Amazon S3 que el CDK de AWS utiliza para almacenar plantillas y activos durante sus procesos de implementación. Para instalar la pila de arranque, ejecute el comando cdk con la acción bootstrap.

> `cdk bootstrap`

**Nota**: Si ejecuta cdk bootstrap sin especificar ninguna opción, se utilizan la cuenta y la región de AWS por defecto. También puedes arrancar un entorno específico especificando una combinación de perfil y cuenta/región. Por ejemplo:

> `cdk bootstrap --profile test 123456789012/us-east-1`

<br>

3. Haz que el CDK de AWS ejecute la plantilla de pila de AWS CloudFormation para implementar la pila. Para ello, desde el directorio raíz del proyecto, ejecute el comando cdk con la acción deploy y el nombre de la pila.

> `cdk deploy HelloCdkStack`

Si tiene éxito, la salida muestra que la pila HelloCdkStack se desplegó sin errores.

**Nota**: Si la salida muestra un mensaje en el que se indica que la pila no define un entorno y que las credenciales de AWS no se han podido obtener de las ubicaciones estándar o no se ha configurado ninguna región, asegúrese de que sus credenciales de AWS están configuradas correctamente en el IDE y, a continuación, vuelva a ejecutar el comando cdk deploy. 

Para confirmar que el tema de Amazon SNS y la cola de Amazon SQS se han desplegado correctamente, envíe un mensaje al tema y, a continuación, compruebe la cola en busca del mensaje recibido. Para ello, puede utilizar el siguiente comando:

> `sns_topic=$(aws sns list-topics --query "Topics[?contains(TopicArn, 'HelloCdkStack')]" --output text)`

> `aws sns publish --subject "Hello from the AWS CDK" --message "This is a message from the AWS CDK." --topic-arn $sns_topic`

<br>

> `sqs_url=$(aws sqs list-queues --output table --query 'QueueUrls[*]' --output text)`

> `aws sqs receive-message --queue-url $sqs_url`


### Limpieza

En este paso, se indica al CDK de AWS que elimine la pila de AWS CloudFormation que implementó en el paso anterior. A continuación, se indica al CDK de AWS que elimine la pila de arranque que instaló en el paso anterior.

> `cdk destroy HelloCdkStack`